In [1]:
from math import cos
from math import sin
from math import pi
from random import shuffle
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

N = 5
citynames = ['Madison', 'Minneapolis', 'Chicago', 'Greenbay', 'Iowacity']
citydistances = np.array([[0,270,150,140,175],
       [270,0,408,280,304],
       [150,408,0,211,221],
       [140,280,211,0,315],
       [175,304,221,315,0]]) # a matrix containing city to city distance

In [2]:
citydistances

array([[  0, 270, 150, 140, 175],
       [270,   0, 408, 280, 304],
       [150, 408,   0, 211, 221],
       [140, 280, 211,   0, 315],
       [175, 304, 221, 315,   0]])

In [3]:
########################################### 
# Genetic Search
ngen = 1000
ngpool = N

gpool = np.zeros((ngpool,N)).astype('int')
 
for i in range(ngpool):
  # We ensure that the first element remains 0
#  if i == 0:
#    gpool[i] = range(N)
#  else:
    gpool[i,1:] = (np.random.permutation(N-1) + 1)

    costmin = N
    tourmin = np.zeros(N)
    cost = np.zeros(ngpool)
for i in range(ngen):
    # step 1. evaluate the fitness of current chromosomes
    # for tsp problem it is the trip length. the smaller the better
    for j in range(ngpool):
        id_shift = gpool[j,-1:].tolist() + gpool[j,:-1].tolist()
        cost[j] = np.sum(np.diag(citydistances[gpool[j,:].tolist(),id_shift]))

    # Record the current best solution
    costmin = np.amin(cost)
    idx = np.argmin(cost)
    tourmin = gpool[idx,:]

    #print(cost)
    #print(costmin)
    # step 2. cross breeding and mutation
    # since each chromosome is an integer vector, cross breeding
    # can not be done simply by combining binary vectors. 
    # Instead, we let the off-spring to keep the common genes of
    # the parents, and randomly shuffle genes when parents disagree
    # for simplicity, we only cross-breed the two best solutions. 

    ridx = sorted(range(len(cost)), key=lambda k: cost[k])


    # parents
    mother = gpool[ridx[0],:]
    father = gpool[ridx[1],:]
    # find index of genes that are the same in father and mother
    sameidx = father == mother
    diffidx = np.nonzero(1 - sameidx)[0]


    if len(diffidx <= 4): # father and mother too close!
      # do mutation
        boy = [0] + (np.random.permutation(N-1) + 1).tolist()
        girl = [0] + (np.random.permutation(N-1) + 1).tolist()
    else:
        boy = father * sameidx
        boy[diffidx] = father[diffidx[np.random.permutation(len(diffidx))]]
        girl = mother * sameidx
        girl[diffidx] = mother[diffidx[np.random.permutation(len(diffidx))]]
  
    #Replace the worst two
    gpool[ridx[-2]] = boy
    gpool[ridx[-1]] = girl

In [4]:
print("cost function evaluation: " + str(ngen*2 + ngpool) + " times!")
print("Minimum trip length = " + str(costmin))
print("optimum tour = " + str(tourmin))

cost function evaluation: 2005 times!
Minimum trip length = 1095.0
optimum tour = [0 2 4 1 3]


In [5]:
citylist = [];
for i in range(N):
    citylist.append(citynames[tourmin[i]])

In [6]:
print("The corresponding cities",citylist)

The corresponding cities ['Madison', 'Chicago', 'Iowacity', 'Minneapolis', 'Greenbay']
